In [1]:
#Importing all the needed libraries
import pandas as pd
import numpy as np 
from pandas.plotting import scatter_matrix

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import Perceptron
from sklearn import metrics


import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline
 

In [3]:
#Loading files with Pandas library using the variable pd
dataSetLOL = 'LeagueOfLedends2017.xlsx'
lol = pd.read_excel(dataSetLOL, encoding='latin1')

#set_option allows me to see all rows and columns of the dataset without truncating them
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



In [4]:
#We have 37404 lines e 98 columns in dataset
lol.shape

(37404, 98)

In [4]:
lol.head()

,gameid,url,league,split,date,week,game,patchno,playerid,side,position,player,team,champion,ban1,ban2,ban3,ban4,ban5,gamelength,result,k,d,a,teamkills,teamdeaths,doubles,triples,quadras,pentas,fb,fbassist,fbvictim,fbtime,kpm,okpm,ckpm,fd,fdtime,teamdragkills,oppdragkills,elementals,oppelementals,firedrakes,waterdrakes,earthdrakes,airdrakes,elders,oppelders,herald,heraldtime,ft,fttime,firstmidouter,firsttothreetowers,teamtowerkills,opptowerkills,fbaron,fbarontime,teambaronkills,oppbaronkills,dmgtochamps,dmgtochampsperminute,dmgshare,earnedgoldshare,wards,wpm,wardshare,wardkills,wcpm,visionwards,visionwardbuys,visiblewardclearrate,invisiblewardclearrate,totalgold,earnedgpm,goldspent,gspd,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,oppgoldat10,gdat10,goldat15,oppgoldat15,gdat15,xpat10,oppxpat10,xpdat10,csat10,oppcsat10,csdat10,csat15,oppcsat15,csdat15
0,1002300127,http://matchhistory.na.leagueoflegends.com/en/...,NALCS,2017-2,42953.7,9.3,3,7.15,1,Blue,Top,Lourlo,Team Liquid,Gnar,LeBlanc,Zac,Shen,Tristana,Kog'Maw,25.383333,0,0,2,0,3,18,0,0,0,0,0,0.0,0,6.73213,0,0.118188,0.827315,0,11.9418,0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,12.368267,0.0,0.0,4,10,0.0,21.536467,0.0,1.0,8725.0,343.729481,0.269631,0.239989,14.0,0.551543,0.184211,2.0,0.078792,5,5.0,,,9067.0,224.749048,9025.0,0.028090,231.0,4.0,3.0,0.0,9.258043,3378.0,3264.0,114.0,5542.0,5419.0,123.0,4669.0,4677.0,-8.0,94.0,89.0,5.0,150.0,136.0,14.0
1,1002300127,http://matchhistory.na.leagueoflegends.com/en/...,NALCS,2017-2,42953.7,9.3,3,7.15,2,Blue,Jungle,Dardoch,Team Liquid,Gragas,LeBlanc,Zac,Shen,Tristana,Kog'Maw,25.383333,0,0,5,1,3,18,0,0,0,0,0,1.0,0,6.73213,0,0.0787919,0.827315,0,11.9418,0,2,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,12.368267,0.0,0.0,4,10,0.0,21.536467,0.0,1.0,4300.0,169.402495,0.132884,0.176005,15.0,0.590939,0.197368,3.0,0.118188,5,6.0,,,7546.0,164.827840,6650.0,-0.241322,15.0,98.0,58.0,5.0,4.451740,2936.0,2883.0,53.0,4467.0,5223.0,-756.0,3134.0,3161.0,-27.0,47.0,47.0,0.0,62.0,79.0,-17.0
2,1002300127,http://matchhistory.na.leagueoflegends.com/en/...,NALCS,2017-2,42953.7,9.3,3,7.15,3,Blue,Middle,Mickey,Team Liquid,Ekko,LeBlanc,Zac,Shen,Tristana,Kog'Maw,25.383333,0,0,3,0,3,18,0,0,0,0,0,0.0,0,6.73213,0,0.315167,0.827315,0,11.9418,0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,12.368267,0.0,0.0,4,10,0.0,21.536467,0.0,1.0,6698.0,263.873933,0.20699,0.257531,16.0,0.630335,0.210526,4.0,0.157584,6,6.0,,,9484.0,241.177150,8700.0,-0.171789,247.0,2.0,1.0,0.0,9.809586,3446.0,3468.0,-22.0,5236.0,6286.0,-1050.0,5088.0,5169.0,-81.0,93.0,92.0,1.0,142.0,143.0,-1.0
3,1002300127,http://matchhistory.na.leagueoflegends.com/en/...,NALCS,2017-2,42953.7,9.3,3,7.15,4,Blue,ADC,Piglet,Team Liquid,Vayne,LeBlanc,Zac,Shen,Tristana,Kog'Maw,25.383333,0,0,3,1,3,18,0,0,0,0,0,1.0,0,6.73213,0,0.157584,0.827315,0,11.9418,0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,12.368267,0.0,0.0,4,10,0.0,21.536467,0.0,1.0,10367.0,408.417597,0.320375,0.197459,10.0,0.393959,0.131579,1.0,0.039396,2,3.0,,,8056.0,184.919764,8275.0,-0.191257,186.0,1.0,1.0,0.0,7.367039,2933.0,3287.0,-354.0,4578.0,5635.0,-1057.0,3141.0,2984.0,157.0,68.0,75.0,-7.0,112.0,128.0,-16.0
4,1002300127,http://matchhistory.na.leagueoflegends.com/en/...,NALCS,2017-2,42953.7,9.3,3,7.15,5,Blue,Support,Matt,Team Liquid,Lulu,LeBlanc,Zac,Shen,Tristana,Kog'Maw,25.383333,0,3,5,0,3,18,1,0,0,0,1,0.0,0,6.73213,0.118188,0.0393959,0.827315,0,11.9418,0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,12.368267,0.0,0.0,4,10,0.0,21.536467,0.0,1.0,2269.0,89.389363,0.0701196,0.129016,21.0,0.827315,0.276316,8.0,0.315167,6,7.0,,,6429.0,120.822587,6880.0,-0.092862,12.0,0.0,0.0,0.0,0.472751,2256.0,2389.0,-133.0,3184.0,4298.0,-1114.0,2593.0,2816.0,-223.0,5.0,2.0,3.0,7.0,2.0,5.0


<h1>Chances of victory: Variables of interest</h1>
<p><h3>We want to check which teams won the match - "result" = 1 at the table -, using as a parameter, variables of interest.</h3></p>
<p>The variables of interest for analyzing the chances of winning for this use case are as follows:</p> 
<ul>
    <li><b>gameID:        </b> Match identifier, unique for each server / league.</li>
    <li><b>side:          </b> Side on which the analyzed team started in the match (Blue or Red).</li>
    <li><b>champion:      </b> Champions chosen in the match by the team.</li>
    <li><b>position:      </b> Position where champions play in the match.</li>
    <li><b>team:          </b> Name of the team in the match.</li>
    <li><b>ft:            </b> Reports whether the team was the first to take down an enemy tower.</li>
</ul>
    <p>However, in order to use the variables for the creation of a predictive model, it is first necessary to work them properly, performing the adequate data cleaning.</p>

<h1>Match operation:</h1>
<p>Due to the nature of how League of Legends matches work, with 5 players for each of the two teams, each player choosing their champion and their position in the game, new columns will be created on a new dataframe identifying which champion each player of the two teams took in the match and what position each took.</p>

<h3>Creating standardization functions</h3>
<p>Below will be created functions whose objective will be to standardize the "team" and "champion" columns, which probably have many invalid values</p>

In [5]:
'''
Here all the champions are being added to a word list 
and they are being cleaned:all characters are being modified to lowercase, eliminating all blanks, eliminating all punctuation,
removing all duplicate words (with the dict). In the end, all the modified words will be added in
a list of champions. This cleaning will be performed through a function called "padronizar_campeoes"
'''
import string

def padronizar_campeoes(champions):
    champions = champions.lower().translate(str.maketrans('', '', string.punctuation)).replace(' ', '').replace('-', '').strip('')
    return champions

In [6]:
'''
Here all the teams are being added to a word list and they are being cleaned: all characters are being modified to lowercase, 
eliminating all blanks, eliminating all punctuation,removing all duplicate words (with the dict). In the end, all the 
modified words will be added in a list of teams. This cleaning will be performed through a function called 
"padronizar_times"
'''
import string

def padronizar_times(teams):
    teams = teams.lower().translate(str.maketrans('', '', string.punctuation)).replace(' ', '').replace('-', '').strip('')
    return teams

<h1>Applying the functions line by line:</h1>
<p>The goal now is to apply these functions line by line, passing the variable we want to change as a parameter. To do this, I will use the "map" method, passing the function name as an argument and assigning it to a new column that will be created with the changed data. The new columns created will have the same name as the original column, but the 's' in front as standard</p>

In [7]:
'''
Applying the standardization function in the "champion" column. A new column will be created and
already standardized, called "s_champion"
'''
lol["s_champion"] = lol["champion"].map(padronizar_campeoes)


In [8]:
'''
Applying the standardization function in the "team" column. A new column will be created and
already standardized, called "s_team"
'''
lol["s_team"] = lol["team"].map(padronizar_times)


<h3>Creating a new dataframe</h3>
<p>To make it easier to use predictive machine learning models and save memory, we don't need the nearly 100 columns in the dataset. Therefore, a new dataframe will be created containing only the necessary columns.</p>

In [9]:
'''
Creating a new variable containing only the variables of interest
'''
w_lol = pd.concat([ lol["s_team"], lol["gameid"], lol["side"], lol["result"],
                   lol['ft'],lol['fbaron'], lol["s_champion"], lol["position"],
                   lol["csdat10"], lol["gdat10"], lol["xpdat10"], lol["gdat15"] ], axis = 1).copy()

'''w_lol = pd.concat([ lol["s_team"], lol["gameid"], lol["side"], lol["result"],
                   lol['ft'],lol['fbaron'], lol["s_champion"], lol["position"],
                   lol["csdat10"], lol["gdat10"], lol["xpdat10"], lol["gdat15"] ], axis = 1).copy()
'''
'''w_lol = pd.concat([ lol["s_team"], lol["gameid"], lol["side"], lol["result"],
                   lol["s_champion"], lol["position"],
                   lol["csdat10"], lol["gdat10"], lol["xpdat10"], lol["gdat15"] ], axis = 1).copy()'''


'w_lol = pd.concat([ lol["s_team"], lol["gameid"], lol["side"], lol["result"],\n                   lol["s_champion"], lol["position"],\n                   lol["csdat10"], lol["gdat10"], lol["xpdat10"], lol["gdat15"] ], axis = 1).copy()'

In [10]:
w_lol.head(10)

,s_team,gameid,side,result,ft,fbaron,s_champion,position,csdat10,gdat10,xpdat10,gdat15
0,teamliquid,1002300127,Blue,0,1.0,0.0,gnar,Top,5.0,114.0,-8.0,123.0
1,teamliquid,1002300127,Blue,0,1.0,0.0,gragas,Jungle,0.0,53.0,-27.0,-756.0
2,teamliquid,1002300127,Blue,0,1.0,0.0,ekko,Middle,1.0,-22.0,-81.0,-1050.0
3,teamliquid,1002300127,Blue,0,1.0,0.0,vayne,ADC,-7.0,-354.0,157.0,-1057.0
4,teamliquid,1002300127,Blue,0,1.0,0.0,lulu,Support,3.0,-133.0,-223.0,-1114.0
5,dignitas,1002300127,Red,1,0.0,1.0,jarvaniv,Top,-5.0,-114.0,8.0,-123.0
6,dignitas,1002300127,Red,1,0.0,1.0,maokai,Jungle,0.0,-53.0,27.0,756.0
7,dignitas,1002300127,Red,1,0.0,1.0,taliyah,Middle,-1.0,22.0,81.0,1050.0
8,dignitas,1002300127,Red,1,0.0,1.0,varus,ADC,7.0,354.0,-157.0,1057.0
9,dignitas,1002300127,Red,1,0.0,1.0,janna,Support,-3.0,133.0,223.0,1114.0


<h3>Standardizing values in strings </h3>
<p> Predictive models only accept numerical values. Therefore, the variables of interest whose values are strings need to be standardized. There are 3 variables in which these problems occur: 'champion', 'side', 'position'. To perform this standardization process, a function was created whose objective was to use the "LabelEncoder" method belonging to the sklearn.preprocessing library. This method was explained on the "StackOverflow" website by a user, at the following link: https://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn <p>

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

class PadronizacaoStrings:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self 

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [ ]:
'''
Here, the Strings standardization function will be called and this process will be assigned to a new
dataframe whose name is "s_lol"
'''
w_lol = (PadronizacaoStrings(columns = ['side', 's_champion', 's_team', 'position']).fit_transform(w_lol)).copy()


In [11]:
w_lol.head()

,s_team,gameid,side,result,ft,fbaron,s_champion,position,csdat10,gdat10,xpdat10,gdat15
0,teamliquid,1002300127,Blue,0,1.0,0.0,gnar,Top,5.0,114.0,-8.0,123.0
1,teamliquid,1002300127,Blue,0,1.0,0.0,gragas,Jungle,0.0,53.0,-27.0,-756.0
2,teamliquid,1002300127,Blue,0,1.0,0.0,ekko,Middle,1.0,-22.0,-81.0,-1050.0
3,teamliquid,1002300127,Blue,0,1.0,0.0,vayne,ADC,-7.0,-354.0,157.0,-1057.0
4,teamliquid,1002300127,Blue,0,1.0,0.0,lulu,Support,3.0,-133.0,-223.0,-1114.0


<h3>Changing the dataframe back to the final shape</h3>
<p>Here is the most essential change to the dataframe. Currently it is structured as follows: Each game has its unique id (gameid). For each game, we have listed 5 players per line, for each of the two teams. Soon after, we have two more lines listing the two teams participating in the match. That is, for each game, we have a total of 12 lines representing each player, their teams and their positions in the game. The change made to the dataframe was to allocate the information contained in these 12 rows to a single row, increasing the number of columns from 6 to 24.</p>

In [12]:
'''
Here is the most essential change to the dataframe. 
'''
i = 0

result = {}

for index, value in w_lol.iterrows():
    if (not(value.gameid in result)):
        result[value.gameid] = {
            "result": value.result,
            "side": value.side,
            "team": value.s_team,
            "gameid": value.gameid,
            #"ft" : value.ft,
            #"fbaron" : value.fbaron
        }
        i = 0
    if i == 10:
        continue
    
    if (i < 5):
        result[value.gameid]["champion"+str(i+1)] = value.s_champion
        result[value.gameid]["position"+str(i+1)] = value.position
        result[value.gameid]["csdif"+str(i+1)] = value.csdat10
        result[value.gameid]["golddif10_"+str(i+1)] = value.gdat10
        result[value.gameid]["xpdif"+str(i+1)] = value.xpdat10
        #result[value.gameid]["golddif15_"+str(i+1)] = value.gdat15
    else:
        result[value.gameid]["enemy_champion"+str(i-4)] = value.s_champion
        result[value.gameid]["enemy_position"+str(i-4)] = value.position

    
    i = i + 1

    
df_lol = pd.DataFrame.from_dict(result, orient='index')


    


In [13]:
'''
Some lines have empty values, but they are not null. Therefore, applying the method of excluding null values will not work.
To solve this problem, I will use a method called "replace", to replace all values with empty spaces in the dataset
into null values. And in the following steps, I will delete all null values.
'''
df_lol.replace('', np.nan, inplace=True)
df_lol.isnull().sum()

result               0
side                 0
team                 0
gameid               0
champion1            0
position1            0
csdif1             534
golddif10_1        534
xpdif1             534
champion2            0
position2            0
csdif2             534
golddif10_2        534
xpdif2             534
champion3            0
position3            0
csdif3             534
golddif10_3        534
xpdif3             534
champion4            0
position4            0
csdif4             534
golddif10_4        534
xpdif4             534
champion5            0
position5            0
csdif5             534
golddif10_5        534
xpdif5             534
enemy_champion1      0
enemy_position1      0
enemy_champion2      0
enemy_position2      0
enemy_champion3      0
enemy_position3      0
enemy_champion4      0
enemy_position4      0
enemy_champion5      0
enemy_position5      0
dtype: int64

<h3>Change in dimensionality:</h3>
    <p>As expected, the dimensionality has changed dramatically. This is because all the lines have been divided into several different pieces. And these pieces were all allocated in 20 new columns representing each position and each champion chosen by each of the 10 players (of the two teams) of the match. </p>

<h3>Checking for null data</h3>

In [ ]:
df_lol.isnull().sum()

In [14]:
'''
As noted above, we have some null values in the dataset, referring to the 'ft' column. The values
nulls represent a total of 17.28% of the dataframe. According to the website
(https://analyticsindiamag.com/5-ways-handle-missing-values-machine-learning-datasets/), if the
null values represent up to a maximum of 30% of the total information in the dataset, it is only safe
delete null values. Therefore, null values will have their lines deleted.

'''
df_lol.dropna(inplace = True)
df_lol.isnull().sum()

result             0
side               0
team               0
gameid             0
champion1          0
position1          0
csdif1             0
golddif10_1        0
xpdif1             0
champion2          0
position2          0
csdif2             0
golddif10_2        0
xpdif2             0
champion3          0
position3          0
csdif3             0
golddif10_3        0
xpdif3             0
champion4          0
position4          0
csdif4             0
golddif10_4        0
xpdif4             0
champion5          0
position5          0
csdif5             0
golddif10_5        0
xpdif5             0
enemy_champion1    0
enemy_position1    0
enemy_champion2    0
enemy_position2    0
enemy_champion3    0
enemy_position3    0
enemy_champion4    0
enemy_position4    0
enemy_champion5    0
enemy_position5    0
dtype: int64

In [15]:
df_lol.shape

(2556, 39)

In [16]:
df_lol.head()

,result,side,team,gameid,champion1,position1,csdif1,golddif10_1,xpdif1,champion2,position2,csdif2,golddif10_2,xpdif2,champion3,position3,csdif3,golddif10_3,xpdif3,champion4,position4,csdif4,golddif10_4,xpdif4,champion5,position5,csdif5,golddif10_5,xpdif5,enemy_champion1,enemy_position1,enemy_champion2,enemy_position2,enemy_champion3,enemy_position3,enemy_champion4,enemy_position4,enemy_champion5,enemy_position5
200035,1,Blue,biggodsjackals,200035,shen,Top,2.0,-35.0,-262.0,zac,Jungle,-11.0,-348.0,-500.0,taliyah,Middle,9.0,580.0,217.0,twitch,ADC,9.0,868.0,486.0,thresh,Support,3.0,293.0,721.0,galio,Top,gragas,Jungle,syndra,Middle,caitlyn,ADC,taric,Support
200036,1,Blue,eunited,200036,jarvaniv,Top,24.0,526.0,503.0,elise,Jungle,-7.0,585.0,-505.0,leblanc,Middle,8.0,640.0,758.0,ashe,ADC,16.0,437.0,554.0,tahmkench,Support,4.0,258.0,472.0,gragas,Top,leesin,Jungle,viktor,Middle,varus,ADC,thresh,Support
200039,0,Blue,goldcoinunited,200039,shen,Top,-2.0,20.0,37.0,elise,Jungle,19.0,325.0,-21.0,syndra,Middle,-6.0,-145.0,-144.0,ashe,ADC,6.0,113.0,179.0,karma,Support,-5.0,-73.0,-60.0,swain,Top,leesin,Jungle,cassiopeia,Middle,caitlyn,ADC,zyra,Support
200048,0,Blue,clgacademy,200048,fiora,Top,13.0,205.0,147.0,olaf,Jungle,13.0,452.0,648.0,leblanc,Middle,-10.0,-267.0,-409.0,jhin,ADC,-13.0,-200.0,-256.0,karma,Support,-11.0,-196.0,181.0,gragas,Top,khazix,Jungle,orianna,Middle,caitlyn,ADC,thresh,Support
200050,1,Blue,tempostorm,200050,galio,Top,-17.0,-390.0,-250.0,elise,Jungle,-5.0,-230.0,-92.0,kassadin,Middle,15.0,338.0,600.0,caitlyn,ADC,-5.0,333.0,-618.0,blitzcrank,Support,11.0,426.0,527.0,shen,Top,leesin,Jungle,leblanc,Middle,ashe,ADC,zyra,Support


<h1>Using OneHotEncoder for categorical data:</h1>

In [17]:
import category_encoders as ce

ce_ohe = ce.OneHotEncoder(handle_unknown='ignore', use_cat_names=True)
s_lol = ce_ohe.fit_transform(df_lol)


In [18]:
df_lol.shape

(2556, 39)

In [19]:
s_lol.shape

(2556, 3068)

<h3>Splitting the Dataframe: </h3>
<p>The division made by train_test_split is random but is done only once. To guarantee the randomness of the process and the equal separation of the dataframe in training and testing, we can divide the dataframe into equal pieces ("folds"), shuffle each piece ("shuffle") and apply the division in each piece. To do this, we need to use the "Kfold" method belonging to the "sklearn.model_selection" library</p>

<h3>Calculating accuracy:</h3>
<p>After the variables of interest are trained, they will be used to try to predict the victory of the match by comparing with the test variables. Accuracy will measure the correctness rate of the variables of interest in forecasting the chances of winning the match. The accuracy calculation will be done for each division made by the kfold method. In the end, the <b> average accuracy of all divisions of the dataframe will be calculated using the kfold method </b>. This will be the final accuracy, having only the variable of interest calculated at the moment as a deterministic parameter for the victory of the match. </p>

In [ ]:
from sklearn.model_selection import KFold


<h3> Testing multiple predictive models based on their performance: </h3>
<p> Before carrying out the processes of calculating the chances of winning according to the variables of interest through the predictive models, it is first necessary to define which is the predictive model that has the best performance for the construction of predictive analyzes. 4 models will be analyzed with all possible variables. However, in the end, only one model will be used for all variables. They are: KNEIGHBORS_CLASSIFIER, RANDOM_FORES_CLASSIFIER, DECISION_TREE_CLASSIFIER, NAIVE_BAYES. </p>

<h3> Cross Validation Score DOES NOT train (fit) the dataset for future predictions </h3>
<p>
    According to a question posted on Stack Overflow (https://stackoverflow.com/questions/42263915/using-sklearn-cross-val-score-and-kfolds-to-fit-and-help-predict-model?rq= 1) the cross_validation_score method is used only to estimate which predicted models have the best performance. This is done by subdividing the training dataset (the original test dataset is not worked on), in several folds (pieces) where 1 of the pieces will be used as a test. This will be done k times (k is passed as a parameter), and in each iteration, 1 new piece of the training dataset will be chosen to test its accuracy. In the end, the average accuracy of the k iterations will be made and the result of the accuracy will be estimated. The original test dataset is not used because it is intended to generalize the forecast for future analysis. We don't want the algorithm to memorize the results of the test dataset. </p>

In [ ]:
df_lol.head()

In [ ]:
'''
The variable X receives all rows of the dataframe and all columns (except the target column "result").
The y variable receives only the "result" column
'''

X = np.array(df_lol.iloc[:,4:])
y = np.array(df_lol['result'])


In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn import model_selection
import warnings
warnings.filterwarnings('ignore')

'''
Here, all predictive models are being tested according to their performance IN ALL VARIABLES
OF INTEREST. In the end, only 1 predictive model will be chosen to be used. The method used
is to divide the dataset into 10 pieces and calculate the accuracy of each piece. In the end we will calculate average
the accuracy of these 10 pieces. The higher the final average, the better the predictive model. Important
point out that the training is not being done here. Only estimates of which models work
best.

'''

models = []
models.append(('RANDOM_FOREST_CLASSIFIER', RandomForestClassifier(n_estimators=500, random_state = 0)))
models.append(('KNEIGHBORS_CLASSIFIER', KNeighborsClassifier()))
models.append(('DECISION_TREE_CLASSIFIER', DecisionTreeClassifier()))
models.append(('NAIVE_BAYS', GaussianNB()))
models.append(('PERCEPTRON', Perceptron()))


# evaluate each model in turn
results = []
names = []
plota_names = []
plota_acuracia = []



for name, model in models:
    kf = KFold(10, shuffle=True, random_state = 1)
    hit_rate = 0
    for training_lines, testing_lines in kf.split(X,y):
       
        kf = model_selection.KFold(10, shuffle=True, random_state = 1)
        X_train, X_test = X[training_lines], X[testing_lines]
        y_train, y_test = y[training_lines], y[testing_lines]

        cv_results = cross_val_score(model, X_train, y_train, cv=kf, scoring='accuracy')
        results.append(cv_results)
        names.append(name)

        print('%s => Accuracy: %f, Standard Deviation: (%f)' % (name, cv_results.mean(), cv_results.std()))

        hit_rate += cv_results.mean()
        
    #plota_names.append(names)
    #plota_acuracia.append(hit_rate/10)

    final_accuracy = hit_rate/10   
    print("Média das 10 acurácias: ",final_accuracy)
    #print("Desvio padrão da média final :",hit_rate.std())
    print()



<h3> Variable of interest with greater influence on the chances of victory </h3>
<p> Which variable of interest has the greatest influence in victories? What are the relationships between the variables of interest? These answers will be given below. </p>

In [ ]:
fields_to_correlate = ['side', 'team', 'champion1', 'position1', 'champion2', 'position2', 'champion3', 'position3', 'champion4', 'position4', 'champion5', 'position5', 'enemy_champion1', 'enemy_position1', 'enemy_champion2', 'enemy_position2', 'enemy_champion3', 'enemy_position3', 'enemy_champion4', 'enemy_position4', 'enemy_champion5', 'enemy_position5']#.split()


In [ ]:
'''
https://towardsdatascience.com/feature-selection-with-pandas-e3690ad8504b

As seen from below code, the optimum number of features is 10. We now feed 10 
as number of features to RFE and get the final set of features given by RFE method, as follows:

'''


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso


#no of features
nof_list=np.arange(1,13)            
high_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)
    model = LinearRegression()
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    score = model.score(X_test_rfe,y_test)
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))

In [ ]:
df_lol.head()

In [ ]:
s_lol.head()

In [ ]:
fields_to_correlate = 'result champion1 champion2 champion3 champion4 champion5'.split()

In [ ]:
def correlation_in_league():
    df_lol.replace(' ', np.nan, inplace=True)
    corr = df_lol[fields_to_correlate].corr()
    print(corr.round(2))

correlation_in_league()

In [ ]:
df_lol['position1'].isnull().sum()

In [ ]:
#df_lol.dropna(inplace = True)
df_lol.isnull().sum()

In [ ]:
plt.figure(figsize=(12,10))
corr = df_lol[fields_to_correlate].corr()
sns.heatmap(corr, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
df_lol.head()

In [ ]:
'''
Criando novo dataframe
'''
'''
lol_ML = pd.concat([ df_lol["result"], df_lol["side"], df_lol["team"],df_lol['gameid'],df_lol["ft"],df_lol['fbaron'],
         df_lol["champion1"], df_lol["position1"],df_lol["csdif1"],df_lol["xpdif1"],df_lol["golddif15_1"],
         df_lol["champion2"], df_lol["position2"],df_lol["csdif2"],df_lol["xpdif2"],df_lol["golddif15_2"], 
         df_lol["champion3"], df_lol["position3"],df_lol["csdif3"],df_lol["xpdif3"],df_lol["golddif15_3"],
         df_lol["champion4"], df_lol["position4"],df_lol["csdif4"],df_lol["xpdif4"],df_lol["golddif15_4"],
         df_lol["champion5"], df_lol["position5"],df_lol["csdif5"],df_lol["xpdif5"],df_lol["golddif15_5"],
         df_lol["enemy_champion1"], df_lol["enemy_champion2"], df_lol["enemy_champion3"],
         df_lol["enemy_champion4"], df_lol["enemy_champion5"]], axis = 1).copy()
'''

lol_ML = pd.concat([ df_lol["result"], df_lol["side"], df_lol["team"],df_lol['gameid'],
         df_lol["champion1"], df_lol["position1"],df_lol["csdif1"],df_lol["xpdif1"],df_lol["golddif10_1"],
         df_lol["champion2"], df_lol["position2"],df_lol["csdif2"],df_lol["xpdif2"],df_lol["golddif10_2"], 
         df_lol["champion3"], df_lol["position3"],df_lol["csdif3"],df_lol["xpdif3"],df_lol["golddif10_3"],
         df_lol["champion4"], df_lol["position4"],df_lol["csdif4"],df_lol["xpdif4"],df_lol["golddif10_4"],
         df_lol["champion5"], df_lol["position5"],df_lol["csdif5"],df_lol["xpdif5"],df_lol["golddif10_5"],
         df_lol["enemy_champion1"], df_lol["enemy_champion2"], df_lol["enemy_champion3"],
         df_lol["enemy_champion4"], df_lol["enemy_champion5"]], axis = 1).copy()



In [20]:
s_lol.head()

result  side_Blue  team_biggodsjackals  team_eunited  \
200035       1          1                    1             0   
200036       1          1                    0             1   
200039       0          1                    0             0   
200048       0          1                    0             0   
200050       1          1                    0             0   

        team_goldcoinunited  team_clgacademy  team_tempostorm  team_deltafox  \
200035                    0                0                0              0   
200036                    0                0                0              0   
200039                    1                0                0              0   
200048                    0                1                0              0   
200050                    0                0                1              0   

        team_redcanids  team_paingaming  team_tshowesports  \
200035               0                0                  0   
200036               0                0                  0   
200039               0                0                  0   
200048               0                0                  0   
200050               0                0                  0   

        team_progamingesports  team_cnbesportsclub  team_intzesports  \
200035                      0                    0                 0   
200036                      0                    0                 0   
200039                      0                    0                 0   
200048                      0                    0                 0   
200050                      0                    0                 0   

        team_teamoneesports  team_keydstars  team_mvp  team_jinairgreenwings  \
200035                    0               0         0                      0   
200036                    0               0         0                      0   
200039                    0               0         0                      0   
200048                    0               0         0                      0   
200050                    0               0         0                      0   

        team_longzhugaming  team_ktrolster  team_ever8winners  \
200035                   0               0                  0   
200036                   0               0                  0   
200039                   0               0                  0   
200048                   0               0                  0   
200050                   0               0                  0   

        team_afreecafreecs  team_samsunggalaxy  team_sktelecomt1  \
200035                   0                   0                 0   
200036                   0                   0                 0   
200039                   0                   0                 0   
200048                   0                   0                 0   
200050                   0                   0                 0   

        team_bbqolivers  team_roxtigers  team_wayispider  team_jteam  \
200035                0               0                0           0   
200036                0               0                0           0   
200039                0               0                0           0   
200048                0               0                0           0   
200050                0               0                0           0   

        team_machi17  team_raisegaming  team_hongkongattitude  \
200035             0                 0                      0   
200036             0                 0                      0   
200039             0                 0                      0   
200048             0                 0                      0   
200050             0                 0                      0   

        team_ahqesportsclub  team_roccat  team_g2esports  team_unicornsoflove  \
200035                    0            0               0                    0   
200036                    0            0               0             

In [34]:
#s_lol.dropna(inplace = True)
s_lol.isnull().sum()

result                          0
side_Blue                       0
team_biggodsjackals             0
team_eunited                    0
team_goldcoinunited             0
team_clgacademy                 0
team_tempostorm                 0
team_deltafox                   0
team_redcanids                  0
team_paingaming                 0
team_tshowesports               0
team_progamingesports           0
team_cnbesportsclub             0
team_intzesports                0
team_teamoneesports             0
team_keydstars                  0
team_mvp                        0
team_jinairgreenwings           0
team_longzhugaming              0
team_ktrolster                  0
team_ever8winners               0
team_afreecafreecs              0
team_samsunggalaxy              0
team_sktelecomt1                0
team_bbqolivers                 0
team_roxtigers                  0
team_wayispider                 0
team_jteam                      0
team_machi17                    0
team_raisegami

In [ ]:
lol_ML.shape

In [ ]:
lol_ML.head()

In [42]:
s_lol.head()

result  side_Blue  team_biggodsjackals  team_eunited  \
200035       1          1                    1             0   
200036       1          1                    0             1   
200039       0          1                    0             0   
200048       0          1                    0             0   
200050       1          1                    0             0   

        team_goldcoinunited  team_clgacademy  team_tempostorm  team_deltafox  \
200035                    0                0                0              0   
200036                    0                0                0              0   
200039                    1                0                0              0   
200048                    0                1                0              0   
200050                    0                0                1              0   

        team_redcanids  team_paingaming  team_tshowesports  \
200035               0                0                  0   
200036               0                0                  0   
200039               0                0                  0   
200048               0                0                  0   
200050               0                0                  0   

        team_progamingesports  team_cnbesportsclub  team_intzesports  \
200035                      0                    0                 0   
200036                      0                    0                 0   
200039                      0                    0                 0   
200048                      0                    0                 0   
200050                      0                    0                 0   

        team_teamoneesports  team_keydstars  team_mvp  team_jinairgreenwings  \
200035                    0               0         0                      0   
200036                    0               0         0                      0   
200039                    0               0         0                      0   
200048                    0               0         0                      0   
200050                    0               0         0                      0   

        team_longzhugaming  team_ktrolster  team_ever8winners  \
200035                   0               0                  0   
200036                   0               0                  0   
200039                   0               0                  0   
200048                   0               0                  0   
200050                   0               0                  0   

        team_afreecafreecs  team_samsunggalaxy  team_sktelecomt1  \
200035                   0                   0                 0   
200036                   0                   0                 0   
200039                   0                   0                 0   
200048                   0                   0                 0   
200050                   0                   0                 0   

        team_bbqolivers  team_roxtigers  team_wayispider  team_jteam  \
200035                0               0                0           0   
200036                0               0                0           0   
200039                0               0                0           0   
200048                0               0                0           0   
200050                0               0                0           0   

        team_machi17  team_raisegaming  team_hongkongattitude  \
200035             0                 0                      0   
200036             0                 0                      0   
200039             0                 0                      0   
200048             0                 0                      0   
200050             0                 0                      0   

        team_ahqesportsclub  team_roccat  team_g2esports  team_unicornsoflove  \
200035                    0            0               0                    0   
200036                    0            0               0             

In [43]:
s_lol.shape

(2556, 3068)

<h3> Training the variables of interest with the chosen model: </h3>
<p> Here, finally, the variables of interest will in fact be trained with the predictive model that fared best in the previous phase and its accuracy will be calculated soon after. This will be done for all variables of interest at the same time, in order to predict the chances of victory. </p>

In [ ]:
#probabilites_variables = np.array(df_lol.iloc[:,4:])
#target = np.array(df_lol['result'])

In [48]:
probabilites_variables = np.array(s_lol.iloc[:,1:])
target = np.array(s_lol['result'])

In [118]:
from sklearn.model_selection import KFold
'''
Here, in fact, the dataset is being trained according to the division made previously between "features" and "target" variables (result). There are several models below, but only
the model that did better during the cross_validation_score tests (better accuracy), will be
used. 
''' 
resultados = []
n_folds = 10
k_folds = KFold(n_folds, shuffle=True, random_state = 1)
scores_teste = 0
scores_treino = 0



for linhas_treino, linhas_teste in k_folds.split(probabilites_variables,target):
    XTreino, XTeste = probabilites_variables[linhas_treino], probabilites_variables[linhas_teste]
    yTreino, yTeste = target[linhas_treino], target[linhas_teste]
    
    # Aqui o melhor modelo será escolhido para treinar as variáveis.    
    #modelr = RandomForestClassifier(                                 
                                    #random_state = 42,
                                    #n_jobs = -1, 
                                    #bootstrap = True, 
                                    #criterion = 'gini',
                                    #max_depth = 50, 
                                    #max_features = 'auto',
                                    #min_samples_leaf = 3, 
                                    #min_samples_split = 10, 
                                    #n_estimators = 100
                                   #)
    #modelo = LogisticRegression(solver='liblinear', multi_class='ovr')
    #modeli = LinearDiscriminantAnalysis()
    #modelk = KNeighborsClassifier(
                                 #algorithm = 'auto', leaf_size = 10, metric = 'minkowski',
                                 #n_neighbors = 10, p = 1, weights = 'distance')
    #modeldtr = DecisionTreeRegressor(max_depth=8, min_samples_leaf=0.13, random_state=3)
    #modeld = DecisionTreeClassifier(random_state = 42,
                                   #criterion = 'gini', max_depth = 2, max_features = 'auto', 
                                   #min_impurity_decrease = 0.2, min_samples_leaf = 3,
                                   #min_samples_split = 8, min_weight_fraction_leaf = 0.2,
                                   #splitter = 'best')
                    
    #modelg = GaussianNB()
    #modelss = StandardScaler()
    #models = SVC(gamma='auto')
    modelp = Perceptron(n_jobs = -1, random_state = 42,
                    penalty = 'l1' ,
                    alpha = 0.0001,
                    max_iter = 3000,
                    class_weight = 'balanced'
               )
    
    
    modelp.fit(XTreino, yTreino)
    previsoes = modelp.predict(XTeste)
    resultados.append(accuracy_score(yTeste, previsoes))
    scores_treino += modelp.score(XTreino, yTreino)
    scores_teste += modelp.score(XTeste, yTeste)
    

    
#Calcula a acurácia    
    
print("Acurácias de cada pedaço dividido pelo dataset =>")
print(resultados)
print("\nMédia de todas as acurácias: ", np.mean(resultados))
print()
print("Média da pontuação de validação do modelo de treino: ", scores_treino/n_folds)
print("Média da pontuação de validação do modelo de teste: ", scores_teste/n_folds)



Acurácias de cada pedaço dividido pelo dataset =>
[0.45703125, 0.6015625, 0.65234375, 0.5546875, 0.5625, 0.63671875, 0.6274509803921569, 0.596078431372549, 0.6274509803921569, 0.45098039215686275]

Média de todas as acurácias:  0.5766804534313726

Média da pontuação de validação do modelo de treino:  0.6158946015909906
Média da pontuação de validação do modelo de teste:  0.5766804534313724


<h1>No tuning</h1>

In [ ]:
Random Forest (sem tunagem e golddif 10 minutos) :
    =>Sem FT e sem FBaron: Média de todas as acurácias:  0.6005720964566931
    =>Com FT e sem FBaron: Média de todas as acurácias:  0.6518608513779529
    =>Com FT e com FBaron: Média de todas as acurácias:  0.7994197482297405


In [ ]:
Random Forest (sem tunagem e golddif 15 minutos):
    =>Sem FT e sem FBaron: Média de todas as acurácias:  0.656142347440945
    =>Com FT e sem FBaron: Média de todas as acurácias:  0.676519438976378
    =>Com FT e com FBaron: Média de todas as acurácias:  0.8087004982953057


<h1>Tuning</h1>

In [64]:

probabilites_variables = np.array(s_lol.iloc[:,1:])
target = np.array(s_lol['result'])


In [65]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(s_lol, target, test_size=0.3, random_state=42)

<h3>Tuning Random Forest.</h3>

In [ ]:
param_grid = { 
    'bootstrap': [True],
    'max_depth': [2, 5, 10, 50, 100, 110],
    'min_samples_leaf': [3, 4, 5, 10],
    'min_samples_split': [ 8, 10, 12],
    'n_estimators': [100, 200, 500],
    'criterion' :['gini', 'entropy'],
    'max_features': ['auto', 'sqrt', 'log2']
}

In [ ]:
from sklearn.model_selection import GridSearchCV

rfc=RandomForestClassifier(random_state=42)
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5, verbose = 1, n_jobs = -1)
CV_rfc.fit(x_train, y_train)


In [ ]:
CV_rfc.best_params_

In [ ]:
CV_rfc.best_score_

In [ ]:
{'bootstrap': True,
 'criterion': 'gini',
 'max_depth': 5,
 'max_features': 'auto',
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 100}
0.7501844717252188

<h3>Tuning Decision Tree Classifier</h3>

In [29]:
param_grid = { 
    'max_depth': [5, 10],
    'criterion' :['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'min_weight_fraction_leaf': [0.2, 0.5],
    'min_impurity_decrease' : [0.2, 0.5],
    'max_features': ['auto', 'sqrt', 'log2']
}

In [30]:
from sklearn.model_selection import GridSearchCV

dtc=DecisionTreeClassifier(random_state=42)
CV_dtc = GridSearchCV(estimator=dtc, param_grid=param_grid, cv= 5, verbose = 1, n_jobs = -1)
CV_dtc.fit(x_train, y_train)

Fitting 5 folds for each of 864 candidates, totalling 4320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   22.1s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   39.1s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:   59.3s
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 4320 out of 4320 | elapsed:  3.3min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_depth': [5, 10], 'criterion': ['gini', 'entropy'], 'splitter': ['best', 'random'], 'min_samples_leaf': [3, 4, 5], 'min_samples_split': [8, 10, 12], 'min_weight_fraction_leaf': [0.2, 0.5], 'min_impurity_decrease': [0.2, 0.5], 'max_features': ['auto', 'sqrt', 'log2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [31]:
CV_dtc.best_params_

{'criterion': 'gini',
 'max_depth': 5,
 'max_features': 'auto',
 'min_impurity_decrease': 0.2,
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'min_weight_fraction_leaf': 0.2,
 'splitter': 'best'}

<h3>Tuning k-nearest neighbor</h3>

In [68]:
param_grid = { 
    'n_neighbors' : [2, 5, 10, 15, 30],
    'weights' : ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [10, 20, 50],
    #'p': [1, 2, 5],
    'metric': ['minkowski', 'euclidean', 'chebyshev']
}

In [ ]:
from sklearn.model_selection import GridSearchCV

knn = KNeighborsClassifier(random_state = 42)
CV_knn = GridSearchCV(estimator=knn, param_grid=param_grid, cv= 5, verbose = 1, n_jobs = -1)
CV_knn.fit(x_train, y_train)

In [ ]:
CV_knn.best_params_

<h1>Tuning Perceptron</h1>

In [114]:
param_grid = { 
    'penalty': ['l1', 'l2', 'elasticnet'],
    'alpha': [0.0001,0.0002,0.0003,0.0005, 0.0010],
    'max_iter': [100, 500, 1000, 3000],
    #'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 5}, {0: 1, 1: 1}, {0: 1, 1: 1}]
}

In [ ]:
from sklearn.model_selection import GridSearchCV

per = Perceptron(random_state = 42)
CV_per = GridSearchCV(estimator=per, param_grid=param_grid, cv= 5, verbose = 1, n_jobs = -1)
CV_per.fit(x_train, y_train)

In [111]:
CV_per.best_params_

{'class_weight': {0: 1, 1: 1}}

In [ ]:
{'penalty': 'l1'}

In [ ]:
{'alpha': 0.0001}

In [ ]:
{'max_iter': 3000}

In [ ]:
{'class_weight': {0: 1, 1: 1}}

In [ ]:
'''

This graph represents the number of times each Champion was chosen in the match.
'''

sortedPicks = w_lol['s_champion']


fig, (ax1) = plt.subplots(ncols=1, sharey=False, figsize=(15,30))
plt.xticks(rotation=90)
sns.countplot(y=sortedPicks, data=w_lol, ax=ax1)
ax1.set_title('Champion Picks')


In [ ]:
w_lol.head()

In [ ]:
w_lol['result'].value_counts()

In [ ]:
'''
List of the 10 most chosen champions in the match
'''
w_lol['s_champion'].value_counts().head(11)


In [ ]:
'''
Counts the number of victories of the 10 most chosen champions
'''
count_gragas = 0
count_varus = 0
count_ashe = 0
count_orianna = 0
count_karma = 0
count_leesin = 0
count_khazix = 0
count_thresh = 0
count_maokai = 0
count_elise = 0

for index, row in w_lol.iterrows():
    if(row.s_champion == 'gragas' and row.result == 1):
        count_gragas += 1
        
    elif(row.s_champion == 'varus' and row.result == 1):
        count_varus += 1
        
    elif(row.s_champion == 'ashe' and row.result == 1):
        count_ashe += 1
        
    elif(row.s_champion == 'orianna' and row.result == 1):
        count_orianna += 1
        
    elif(row.s_champion == 'karma' and row.result == 1):
        count_karma += 1
        
    elif(row.s_champion == 'leesin' and row.result == 1):
        count_leesin += 1
        
    elif(row.s_champion == 'khazix' and row.result == 1):
        count_khazix += 1
        
    elif(row.s_champion == 'thresh' and row.result == 1):
        count_thresh += 1
        
    elif(row.s_champion == 'maokai' and row.result == 1):
        count_maokai += 1
        
    elif(row.s_champion == 'elise' and row.result == 1):
        count_elise += 1
        
        
        


In [ ]:
'''
Counts the number of defeats of the 10 most chosen champions
'''
countd_gragas = 0
countd_varus = 0
countd_ashe = 0
countd_orianna = 0
countd_karma = 0
countd_leesin = 0
countd_khazix = 0
countd_thresh = 0
countd_maokai = 0
countd_elise = 0

for index, row in w_lol.iterrows():
    if(row.s_champion == 'gragas' and row.result == 0):
        countd_gragas += 1
        
    elif(row.s_champion == 'varus' and row.result == 0):
        countd_varus += 1
        
    elif(row.s_champion == 'ashe' and row.result == 0):
        countd_ashe += 1
        
    elif(row.s_champion == 'orianna' and row.result == 0):
        countd_orianna += 1
        
    elif(row.s_champion == 'karma' and row.result == 0):
        countd_karma += 1
        
    elif(row.s_champion == 'leesin' and row.result == 0):
        countd_leesin += 1
        
    elif(row.s_champion == 'khazix' and row.result == 0):
        countd_khazix += 1
        
    elif(row.s_champion == 'thresh' and row.result == 0):
        countd_thresh += 1
        
    elif(row.s_champion == 'maokai' and row.result == 0):
        countd_maokai += 1
        
    elif(row.s_champion == 'elise' and row.result == 0):
        countd_elise += 1
        
        
        


In [ ]:
'''
Number of defeats and victories of the most chosen champions
'''

print("Gragas  => Derrotas: {}, Vitórias: {}".format(count_gragas, countd_gragas))
print("Varus   => Derrotas: {}, Vitórias: {}".format(count_varus, countd_varus))
print("Ashe    => Derrotas: {}, Vitórias: {}".format(count_ashe, countd_ashe))
print("Orianna => Derrotas: {}, Vitórias: {}".format(count_orianna, countd_orianna))
print("Karma   => Derrotas: {}, Vitórias: {}".format(count_karma, countd_karma))
print("Leesin  => Derrotas: {}, Vitórias: {}".format(count_leesin, countd_leesin))
print("Khaziz  => Derrotas: {}, Vitórias: {}".format(count_khazix, countd_khazix))
print("Thresh  => Derrotas: {}, Vitórias: {}".format(count_thresh, countd_thresh))
print("Maokai  => Derrotas: {}, Vitórias: {}".format(count_maokai, countd_maokai))
print("Elise   => Derrotas: {}, Vitórias: {}".format(count_elise, countd_elise))